Chinook Business Analysis Using SQL
===

For this project we will use a SQLite database file called chinook.db. The database contains information about a fictional digital music shop, it is like iTunes Store but kind of like a mini version of it.

The Chinook database contains information about the artists, songs, shop's employees, customers, and the customers purchases. The database has 11 tables to store all these information.

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Overview
---

First, let's see our database.

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Selecting Albums to Purchase
---

Our first mission is to find out which genres sell the most tracks in the USA, then select the best three of them.

In [15]:
%%sql
SELECT g.name genre_name, COUNT(c.country) n_of_times_sold
FROM customer c
LEFT JOIN invoice i ON i.customer_id = c.customer_id
LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
LEFT JOIN track t ON t.track_id = il.track_id
LEFT JOIN genre g ON g.genre_id = t.genre_id
WHERE country = "USA"
GROUP BY 1
ORDER BY 2 DESC;

Done.


genre_name,n_of_times_sold
Rock,561
Alternative & Punk,130
Metal,124
R&B/Soul,53
Blues,36
Alternative,35
Latin,22
Pop,22
Hip Hop/Rap,20
Jazz,14


From the table above, we can see that Rock, Alternative & Punk and Metal genres sold the most tracks in the USA.

The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store and we have the following information about them:

|Artist | Name Genre |
|--------------------|
|Regal |Hip-Hop |
|Red Tone |	Punk |
|Meteor and the Girls |	Pop |
|Slim Jim Bites |Blues |

Based on our findings above, I would recommend Red Tone, Meteor and the Girls and Slim Jim Bites.

Here is another solution with sales percentages:

In [71]:
%%sql

WITH usa_tracks_sold AS
   (SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA")

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    ROUND(cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold),2) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.53
Alternative & Punk,130,0.12
Metal,124,0.12
R&B/Soul,53,0.05
Blues,36,0.03
Alternative,35,0.03
Latin,22,0.02
Pop,22,0.02
Hip Hop/Rap,20,0.02
Jazz,14,0.01


Analyzing Employee Sales Performance
---

Next, we are going to find out the total dollar amount of sales assigned to each sales support agent within the company.

In [34]:
%%sql
SELECT
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    ROUND(SUM(i.total), 2) total_amt
FROM invoice i
JOIN customer c ON c.customer_id = i.customer_id
JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY 1,2;

Done.


employee,hire_date,total_amt
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


It looks like Jane did the most sales followed by Margaret and Steve. However, if we look at the hiring dates, Jane was the first one that hired followed by Margaret and Steve. It makes sense that Jane did higher sales than Margaret and Steve. There isn't also a huge gap between the total dollar amount of sales they did, so we can't make any clear assumptions for employee performances.

Analyzing Sales by Country
---

Next, we will analyze the sales data for customers from each different country. We will find:

* total number of customers
* total value of sales
* average value of sales per customer
* average order value

for each country.

In [69]:
%%sql
SELECT 
    c.country, 
    COUNT(DISTINCT c.customer_id) purchases, 
    ROUND(SUM(i.total),2) total_sales, 
    ROUND(AVG(i.total),2) ave_sales    
FROM customer c
JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


country,purchases,total_sales,ave_sales
USA,13,1040.49,7.94
Canada,8,535.59,7.05
Brazil,5,427.68,7.01
France,5,389.07,7.78
Germany,4,334.62,8.16
United Kingdom,3,245.52,8.77
Czech Republic,2,273.24,9.11
India,2,183.15,8.72
Portugal,2,185.13,6.38
Argentina,1,39.6,7.92


Here is another solution from a different point of view:

In [70]:
%%sql

WITH country_or_other AS
    (SELECT
       CASE
           WHEN (SELECT count(*)
                 FROM customer
                 where country = c.country) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id)

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (SELECT
        country,
        count(distinct customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price) / count(distinct customer_id) customer_lifetime_value,
        SUM(unit_price) / count(distinct invoice_id) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC);

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


Based on the data, there may be opportunity in the following countries:

* Czech Republic
* United Kingdom
* India

Albums vs Individual Tracks
---

Next, we will categorize each invoice as either an album purchase or not, and calculate the following statistics:
* Number of invoices
* Percentage of invoices

In [65]:
%%sql

WITH invoice_first_track AS
    (SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1)

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) percent
FROM
    (SELECT ifs.*, CASE WHEN
                 (SELECT t.track_id FROM track t
                  WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id) IS NULL
             AND
                 (SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs)
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


We can see that 81% of the purchases are not album purchases, and about 19% of them are album purchases. We can says that people are more willing to buy a single than a whole album.

Which artist is used in the most playlists?
---

In [31]:
%%sql
select ar.name "Artist Name", COUNT(DISTINCT pl.name) "Playlist Count"
from track t
join album al on al.album_id = t.album_id
join artist ar on ar.artist_id = al.artist_id
join playlist_track plt on plt.track_id = t.track_id
join playlist pl on pl.playlist_id = plt.playlist_id
group by 1
order by 2 desc
limit 10;

Done.


Artist Name,Playlist Count
Eugene Ormandy,6
Academy of St. Martin in the Fields & Sir Neville Marriner,5
Berliner Philharmoniker & Herbert Von Karajan,5
English Concert & Trevor Pinnock,5
The King's Singers,5
"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",4
Adrian Leaper & Doreen de Feis,4
Alberto Turco & Nova Schola Gregoriana,4
Antal Doráti & London Symphony Orchestra,4
Barry Wordsworth & BBC Concert Orchestra,4


How many tracks have been purchased vs not purchased?
---

In [55]:
%%sql
select 
    count(distinct track_id) as purchased,
    (SELECT count(*) from track)- count(distinct track_id) as not_puchased
from invoice_line

Done.


purchased,not_puchased
1806,1697


Do protected vs non-protected media types have an effect on popularity?
---

In [56]:
%sql select * from media_type

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


In [62]:
%%sql
select 
    CASE WHEN m.name LIKE "Protected%" THEN "protected" 
    ELSE "not" END AS "is_protected",
    count(il.invoice_id) as count
from track t
left join invoice_line il on t.track_id = il.track_id
left join media_type m on m.media_type_id = t.media_type_id
group by 1;

Done.


is_protected,count
not,4315
protected,442


It looks like tracks that are not protected are much more popular.

Conclusion
---

For this project, we did a detailed analysis on Chinook data and found out important information about purchases.